### IMPORTS

In [1]:
import json
from web3 import Web3

### LOAD SETTINGS & LATEST ABI FILE

In [2]:
with open('settings.json') as json_file:
    settings = json.load(json_file)

In [3]:
with open('latest.json') as json_file:
    latest = json.load(json_file)

### IPFS & BLOCKCHAIN GATEWAY CONNECTIONS

In [4]:
class gateway:
    def __init__(self, foo):
        self.host = foo['host']
        self.port = foo['port']

In [5]:
blockchain = gateway(settings['gateways']['blockchain'])

In [6]:
ipfs = gateway(settings['gateways']['ipfs'])

### BLOCKCHAIN PUBLIC/PRIVATE KEYS

In [7]:
class keypair:
    def __init__(self, foo):
        self.public = foo['public']
        self.private = foo['private']

In [8]:
keys = keypair(settings['keys'])

### CONNECT TO CHAIN VIA WEBSOCKET

In [9]:
web3 = Web3(Web3.WebsocketProvider('ws://' + blockchain.host + ':' + blockchain.port))

In [10]:
web3.isConnected()

True

### GENERATE USABLE CONTRACTS

In [78]:
def contract(block):
    return web3.eth.contract(
        address = block['address'],
        abi = block['abi']
    )

In [79]:
class manager_contracts:
    def __init__(self):
        
        # USER MANAGER
        self.user = contract(latest['usermanager'])
        
        # DEVICE MANAGER
        self.device = contract(latest['devicemanager'])
        
        # TASK MANAGER
        self.task = contract(latest['taskmanager'])
        
        # TOKEN MANAGER
        self.token = contract(latest['tokenmanager'])

In [80]:
managers = manager_contracts()

### TEST FETCHING

In [82]:
managers.token.functions.price().call()

0